In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from tqdm.notebook import tqdm

In [2]:
h = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}

In [3]:
res = req.get("https://www.melon.com/chart/index.htm",headers = h)
res

<Response [200]>

In [4]:
import requests

In [5]:
url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
res = requests.get(url, headers=req_header_dict)
print(res.status_code)

200 # 정상

200


200

In [6]:
if res.ok:
    html = res.text
    #html 전체 소스보기
    #print(html)
    soup = BeautifulSoup(html, 'html.parser')
    #html parsing
    #print(soup)

In [7]:
#parsing한 html중 곡 에대한 정보를 담음 태그만 조회!!
a_tags = soup.select("div#tb_list tr a[href*='playSong']")
#print(a_tags)

In [8]:
import re

In [9]:
song_list = []
for idx, a_tag in enumerate(a_tags,1):
    #노래 1곡의 정보를 저장할 dict 선언 
    song_dict = {}
    #노래 제목 갖고오기 <a href="">노래제목</a>
    song_title = a_tag.text
    song_dict['song_title'] = song_title
    
    #print(song_dict)
    
    #a태그의 href 속성의 값을 추출하기 javascript:melon.play.playSong('1000002721',34535898);
    href_value = a_tag['href']

    #Song ID를 찾기 위한 정규표현식
    matched = re.search(r'(\d+)\);', href_value)
    #print(matched)
    if matched:
        song_id = matched.group(1) # group(0) : 34535898);  group(1) : 34535898
        song_dict['song_id'] = song_id
        #print(song_id)
        
        song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        song_dict['song_detail_url'] = song_detail_url
        song_list.append(song_dict)

print(len(song_list))    
print(song_list[:3])

100
[{'song_title': 'Perfect Night', 'song_id': '36910957', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=36910957'}, {'song_title': 'Drama', 'song_id': '36956630', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=36956630'}, {'song_title': 'Baddie', 'song_id': '36871671', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=36871671'}]


In [10]:
song_detail_list = []
for idx, song in enumerate(song_list[:3],1): # 결과가 매번 100개씩출력되어서 3순위까지 임시로 슬라이싱
    #노래 1곡의 상세정보를 저장할 dict
    song_detail_dict = {}
    song_detail_url = song['song_detail_url']

    
    res = requests.get(song_detail_url, headers=req_header_dict)
    #print(res.status_code)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        #print(idx,song['song_title'])
        song_detail_dict['곡명'] = song['song_title']
        #노래 1곡의 상세정보에 '곡명'을 찾아 주었다. 
        #다음으로 상세정보 '가수'를 찾는다
        singer = soup.select("a[href*='goArtistDetail'] span")
        #print(singer)
        if singer:
            song_detail_dict['가수'] = singer[0].text # singer리스트에서 0번째 순서에 해당하는것을 text로 뽑으면 이름이다.
        #print(singer[0].text)
        
        #다음으로 div.meta에서 dd로 끝나는 값들을 넣어준다
        song_di = soup.select("div.meta dd")
        #print(song_di)
        if song_di:
            song_detail_dict['앨범'] = song_di[0].text
            song_detail_dict['발매일'] = song_di[1].text
            song_detail_dict['장르'] = song_di[2].text
           
            
        song_detail_list.append(song_detail_dict)   
print(song_detail_list)

[{'곡명': 'Perfect Night', '가수': 'LE SSERAFIM (르세라핌)', '앨범': 'Perfect Night', '발매일': '2023.10.27', '장르': '댄스'}, {'곡명': 'Drama', '가수': 'aespa', '앨범': 'Drama - The 4th Mini Album', '발매일': '2023.11.10', '장르': '댄스'}, {'곡명': 'Baddie', '가수': 'IVE (아이브)', '앨범': "I'VE MINE", '발매일': '2023.10.13', '장르': '댄스'}]


In [11]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

# song_list 생성 코드는 그대로 유지

song_detail_list = []
for idx, song in enumerate(song_list[:100], 1):
    # 노래 1곡의 상세정보를 저장할 dict
    song_detail_dict = {}
    song_detail_url = song['song_detail_url']

    # 곡 ID를 추출하기
    song_id = re.search(r'songId=(\d+)', song_detail_url).group(1)
    song_detail_dict['곡 ID'] = song_id

    res = req.get(song_detail_url, headers=req_header_dict)
    if res.ok:
        soup = bs(res.text, 'html.parser')
        song_detail_dict['곡명'] = song['song_title']

        # 가수 정보 추출
        singer = soup.select("a[href*='goArtistDetail'] span")
        if singer:
            song_detail_dict['가수'] = singer[0].text

        # 앨범, 발매일, 장르 정보 추출
        song_di = soup.select("div.meta dd")
        if song_di:
            song_detail_dict['앨범'] = song_di[0].text
            song_detail_dict['발매일'] = song_di[1].text
            song_detail_dict['장르'] = song_di[2].text

        # song_detail_dict을 song_detail_list에 추가
        song_detail_list.append(song_detail_dict)

# 곡 ID를 포함하여 출력
for song_detail in song_detail_list:
    print(f"곡 ID: {song_detail['곡 ID']}, 곡명: {song_detail['곡명']}, 가수: {song_detail.get('가수', 'N/A')}, 앨범: {song_detail.get('앨범', 'N/A')}, 발매일: {song_detail.get('발매일', 'N/A')}, 장르: {song_detail.get('장르', 'N/A')}")

곡 ID: 36910957, 곡명: Perfect Night, 가수: LE SSERAFIM (르세라핌), 앨범: Perfect Night, 발매일: 2023.10.27, 장르: 댄스
곡 ID: 36956630, 곡명: Drama, 가수: aespa, 앨범: Drama - The 4th Mini Album, 발매일: 2023.11.10, 장르: 댄스
곡 ID: 36871671, 곡명: Baddie, 가수: IVE (아이브), 앨범: I'VE MINE, 발매일: 2023.10.13, 장르: 댄스
곡 ID: 37003911, 곡명: To. X, 가수: 태연 (TAEYEON), 앨범: To. X - The 5th Mini Album, 발매일: 2023.11.27, 장르: R&B/Soul
곡 ID: 36382580, 곡명: 헤어지자 말해요, 가수: 박재정, 앨범: 1집 Alone, 발매일: 2023.04.20, 장르: 발라드
곡 ID: 36713849, 곡명: Love Lee, 가수: AKMU (악뮤), 앨범: Love Lee, 발매일: 2023.08.21, 장르: 댄스
곡 ID: 34451383, 곡명: 인사, 가수: 범진, 앨범: 인사, 발매일: 2021.12.24, 장르: 발라드, 인디음악
곡 ID: 36617841, 곡명: Seven (feat. Latto) - Clean Ver., 가수: 정국, 앨범: Seven (feat. Latto) - Clean Ver., 발매일: 2023.07.14, 장르: 댄스
곡 ID: 36852553, 곡명: You & Me, 가수: 제니 (JENNIE), 앨범: JENNIE Special Single [You & Me], 발매일: 2023.10.06, 장르: 댄스
곡 ID: 36713850, 곡명: 후라이의 꿈, 가수: AKMU (악뮤), 앨범: Love Lee, 발매일: 2023.08.21, 장르: 포크/블루스
곡 ID: 4352438, 곡명: 첫 눈, 가수: EXO, 앨범: 겨울 스페셜 앨범 '12월의 기적 (Miracles

In [13]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

# song_list 생성 코드는 그대로 유지

song_detail_list = []
for idx, song in enumerate(song_list[:100], 1):
    # 노래 1곡의 상세정보를 저장할 dict
    song_detail_dict = {}
    song_detail_url = song['song_detail_url']

    # 곡 ID를 추출하기
    song_id = re.search(r'songId=(\d+)', song_detail_url).group(1)
    song_detail_dict['곡 ID'] = song_id

    res = req.get(song_detail_url, headers=req_header_dict)
    if res.ok:
        soup = bs(res.text, 'html.parser')
        song_detail_dict['곡명'] = song['song_title']

        # 가수 정보 추출
        singer = soup.select("a[href*='goArtistDetail'] span")
        if singer:
            song_detail_dict['가수'] = singer[0].text

        # 앨범, 발매일, 장르 정보 추출
        song_di = soup.select("div.meta dd")
        if song_di:
            song_detail_dict['앨범'] = song_di[0].text
            song_detail_dict['발매일'] = song_di[1].text
            song_detail_dict['장르'] = song_di[2].text

        # song_detail_dict을 song_detail_list에 추가
        song_detail_list.append(song_detail_dict)

# 데이터프레임으로 변환
df = pd.DataFrame(song_detail_list)

# CSV 파일로 저장
df.to_csv('melon_chart_data.csv', index=False)

# 파일을 특정 경로에 저장하기
file_path = 'C:/Users/JY/Desktop/bigdaeseol/melon_chart_data.csv'
df.to_csv(file_path, index=False)
print(f"데이터가 성공적으로 {file_path}에 저장되었습니다.")

데이터가 성공적으로 C:/Users/JY/Desktop/bigdaeseol/melon_chart_data.csv에 저장되었습니다.


In [47]:
import pandas as pd
import pymysql

# MySQL 접속 정보
db_username = 'root'
db_password = 'k@ng1llkoo'
db_host = '127.0.0.1'
db_port = '3306'
db_name = 'melon_top_100'

# CSV 파일에서 데이터프레임으로 읽어오기
df = pd.read_csv('melon_chart_data.csv')

# MySQL 연결
conn = pymysql.connect(
    host=db_host,
    port=int(db_port),
    user=db_username,
    password=db_password,
    database=db_name,
    charset='utf8mb4'
)

# 테이블 생성 쿼리
create_table_query = """
CREATE TABLE IF NOT EXISTS song (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    artist VARCHAR(255),
    genre VARCHAR(255),
    album VARCHAR(255)
);
"""

# 테이블 생성
with conn.cursor() as cursor:
    cursor.execute(create_table_query)
conn.commit()

# 데이터 추가 쿼리
insert_query = "INSERT INTO song (id, title, artist, genre, album) VALUES (%s, %s, %s, %s, %s)"

# 각 행을 순회하며 데이터 추가
with conn.cursor() as cursor:
    for index, row in df.iterrows():
        song_id = row.get('곡 ID', 0)
        try:
            song_id = int(song_id)
        except ValueError:
            song_id = 0
        cursor.execute(insert_query, (song_id, row['곡명'], row.get('가수', ''), row.get('장르', ''), row.get('앨범', '')))

# 변경사항 저장
conn.commit()

print("데이터가 MySQL 테이블에 성공적으로 저장되었습니다.")

데이터가 MySQL 테이블에 성공적으로 저장되었습니다.


In [90]:
import pandas as pd
from selenium import webdriver
import time
from tqdm.notebook import tqdm

In [91]:
path = "C:/Users/JY/chromedriver-win64/chromedriver"
driver = webdriver.Chrome(path)

driver.get('http://www.melon.com')
time.sleep(2)

C:\Users\JY\AppData\Local\Temp/ipykernel_8088/2450207543.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [92]:
driver.find_element_by_css_selector(".menu_bg.menu01").click()
time.sleep(1)
driver.find_element_by_css_selector(".menu_chart.m1").click()

C:\Users\JY\AppData\Local\Temp/ipykernel_8088/4211524388.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(".menu_bg.menu01").click()
C:\Users\JY\AppData\Local\Temp/ipykernel_8088/4211524388.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(".menu_chart.m1").click()


In [38]:
more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
if more_info_list:
    more_info_list[0].click()  # 첫 번째 요소 클릭
else:
    print("No elements found.")

C:\Users\JY\AppData\Local\Temp/ipykernel_8088/3716412134.py:1: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")


In [39]:
song = driver.find_element_by_css_selector(".song_name")
song.text

C:\Users\JY\AppData\Local\Temp/ipykernel_8088/642279601.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  song = driver.find_element_by_css_selector(".song_name")


'Perfect Night'

In [40]:
artist = driver.find_element_by_css_selector(".artist")
artist.text

C:\Users\JY\AppData\Local\Temp/ipykernel_8088/3085000380.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  artist = driver.find_element_by_css_selector(".artist")


'LE SSERAFIM (르세라핌)'

In [41]:
meta_info = driver.find_element_by_css_selector(".list").text.split('\n')
meta_info

C:\Users\JY\AppData\Local\Temp/ipykernel_8088/1867688605.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  meta_info = driver.find_element_by_css_selector(".list").text.split('\n')


['앨범',
 'Perfect Night',
 '발매일',
 '2023.10.27',
 '장르',
 '댄스',
 'FLAC',
 'Flac 16/24bit']

In [42]:
meta_info[1]

'Perfect Night'

In [43]:
meta_info[3]

'2023.10.27'

In [44]:
like_count = driver.find_element_by_id("d_like_count")
like_count.text

C:\Users\JY\AppData\Local\Temp/ipykernel_8088/1776764269.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  like_count = driver.find_element_by_id("d_like_count")


'67,495'

In [45]:
artist_link = driver.find_element_by_partial_link_text("LE SSERAFIM (르세라핌)")
if artist_link:
    artist_link.click()
    time.sleep(2)  # 페이지 이동을 기다리기 위해 잠시 대기
    print("LE SSERAFIM (르세라핌) 아티스트 페이지로 이동했습니다.")
else:
    print("LE SSERAFIM (르세라핌) 아티스트 링크를 찾을 수 없습니다.")

C:\Users\JY\AppData\Local\Temp/ipykernel_8088/120039003.py:1: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  artist_link = driver.find_element_by_partial_link_text("LE SSERAFIM (르세라핌)")


LE SSERAFIM (르세라핌) 아티스트 페이지로 이동했습니다.


In [46]:
artist_info_dl = driver.find_element_by_css_selector("dl.atist_info.clfix")
debut_date_element = artist_info_dl.find_element_by_xpath("//dt[text()='데뷔']/following-sibling::dd/span[@class='gubun']")
debut_date = debut_date_element.text.strip()
print(f"데뷔 날짜: {debut_date}")

activity_type_element = artist_info_dl.find_element_by_xpath("//dt[text()='활동유형']/following-sibling::dd")
activity_type = activity_type_element.text.strip()
print(f"활동 유형: {activity_type}")

데뷔 날짜: 2022.05.02
활동 유형: 그룹


C:\Users\JY\AppData\Local\Temp/ipykernel_8088/2539826091.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  artist_info_dl = driver.find_element_by_css_selector("dl.atist_info.clfix")
C:\Users\JY\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  warnings.warn("find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead")


In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--mute-audio")  # 음소거 옵션 추가
chrome_options.add_argument("--disable-audio-output")  # 오디오 출력 비활성화 옵션 추가


driver_path = "C:/Users/JY/chromedriver-win64/chromedriver"
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)

driver.get("https://www.melon.com/chart/index.htm")

# ...

# 각 곡의 상세 정보를 크롤링
for idx, song_detail in enumerate(song_detail_list[:22], 1):
    # 가수 정보가 있는 경우에만 진행
    if '가수' in song_detail:
        artist_name = song_detail['가수']

        # 가수 상세 페이지로 이동
        artist_link = driver.find_element_by_partial_link_text(artist_name)
        if artist_link:
            artist_link.click()
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "dl.atist_info.clfix")))
            
            # 아티스트 정보 크롤링
            artist_info_dl = driver.find_element_by_css_selector("dl.atist_info.clfix")

            try:
                # 아티스트 고유 ID 추출
                artist_id_match = re.search(r"goArtistDetail\('(\d+)", driver.page_source)
                if artist_id_match:
                    artist_id = artist_id_match.group(1)
                    print(f"{artist_name}의 아티스트 ID: {artist_id}")
                    song_detail['아티스트 ID'] = artist_id
                else:
                    print(f"{artist_name}의 아티스트 ID를 찾을 수 없습니다.")
                    song_detail['아티스트 ID'] = 'N/A'
                
                debut_date_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='데뷔']/following-sibling::dd/span[@class='gubun']"))
                )
                debut_date = debut_date_element.text.strip()
                print(f"데뷔 날짜: {debut_date}")
                
                # 데뷔 날짜를 song_detail에 추가
                song_detail['데뷔 날짜'] = debut_date
            except Exception as e:
                song_detail['아티스트 ID'] = 'N/A'
                song_detail['데뷔 날짜'] = 'N/A'
                print(f"데뷔일: N/A")

            try:
                activity_type_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='활동유형']/following-sibling::dd"))
                )
                activity_type = activity_type_element.text.strip()
                print(f"활동 유형: {activity_type}")
                
                # 활동 유형을 song_detail에 추가
                song_detail['활동 유형'] = activity_type
            except Exception as e:
                print(f"활동 유형을 가져오는 중 에러 발생: {e}")
                song_detail['활동 유형'] = 'N/A'

            try:
                agency_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='소속사']/following-sibling::dd"))
                )
                agency = agency_element.text.strip()
                print(f"소속사: {agency}")
                
                # 소속사를 song_detail에 추가
                song_detail['소속사'] = agency
            except Exception as e:
                print(f"소속사를 가져오는 중 에러 발생: {e}")
                song_detail['소속사'] = 'N/A'
                
            disable_player_script = """
            var playButtons = document.querySelectorAll('a[href^="javascript:melon.play.playSong"]'); 
            playButtons.forEach(function(button) {
                var songIdMatch = button.getAttribute('href').match(/\('(\d+)'/);
                if (songIdMatch && (songIdMatch[1] === '36699489' || songIdMatch[1] === '36595401')) {
                    button.removeAttribute('href');
                    button.style.pointerEvents = 'none';
                }
            });
            """
            driver.execute_script(disable_player_script)

            # 아티스트 페이지에서 뒤로 돌아가기
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "service_list_song")))

# 결과를 출력
for song_detail in song_detail_list:
    print(f"데뷔 날짜: {song_detail.get('데뷔 날짜', 'N/A')}, 활동 유형: {song_detail.get('활동 유형', 'N/A')}, 아티스트 ID: {song_detail.get('아티스트 ID', 'N/A')}, 소속사: {song_detail.get('소속사', 'N/A')}")

C:\Users\JY\AppData\Local\Temp/ipykernel_16756/472664912.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
C:\Users\JY\AppData\Local\Temp/ipykernel_16756/472664912.py:26: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  artist_link = driver.find_element_by_partial_link_text(artist_name)
C:\Users\JY\AppData\Local\Temp/ipykernel_16756/472664912.py:32: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  artist_info_dl = driver.find_element_by_css_selector("dl.atist_info.clfix")


LE SSERAFIM (르세라핌)의 아티스트 ID: 3092950
데뷔 날짜: 2022.05.02
활동 유형: 그룹
소속사: (주)쏘스뮤직
aespa의 아티스트 ID: 2899555
데뷔 날짜: 2020.11.17
활동 유형: 그룹
소속사: (주)SM엔터테인먼트
IVE (아이브)의 아티스트 ID: 3055146
데뷔 날짜: 2021.12.01
활동 유형: 그룹
소속사: 스타쉽 엔터테인먼트
태연 (TAEYEON)의 아티스트 ID: 236797
데뷔 날짜: 2015.10.07
활동 유형: 솔로
소속사: (주)SM엔터테인먼트
정국의 아티스트 ID: 725987
데뷔일: N/A
활동 유형을 가져오는 중 에러 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF6E95182B2+55298]
	(No symbol) [0x00007FF6E9485E02]
	(No symbol) [0x00007FF6E93405AB]
	(No symbol) [0x00007FF6E9320038]
	(No symbol) [0x00007FF6E93A6BC7]
	(No symbol) [0x00007FF6E93BA15F]
	(No symbol) [0x00007FF6E93A1E83]
	(No symbol) [0x00007FF6E937670A]
	(No symbol) [0x00007FF6E9377964]
	GetHandleVerifier [0x00007FF6E9890AAB+3694587]
	GetHandleVerifier [0x00007FF6E98E728E+4048862]
	GetHandleVerifier [0x00007FF6E98DF173+4015811]
	GetHandleVerifier [0x00007FF6E95B47D6+695590]
	(

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF6E95182B2+55298]
	(No symbol) [0x00007FF6E9485E02]
	(No symbol) [0x00007FF6E93405AB]
	(No symbol) [0x00007FF6E9320038]
	(No symbol) [0x00007FF6E93A6BC7]
	(No symbol) [0x00007FF6E93BA15F]
	(No symbol) [0x00007FF6E93A1E83]
	(No symbol) [0x00007FF6E937670A]
	(No symbol) [0x00007FF6E9377964]
	GetHandleVerifier [0x00007FF6E9890AAB+3694587]
	GetHandleVerifier [0x00007FF6E98E728E+4048862]
	GetHandleVerifier [0x00007FF6E98DF173+4015811]
	GetHandleVerifier [0x00007FF6E95B47D6+695590]
	(No symbol) [0x00007FF6E9490CE8]
	(No symbol) [0x00007FF6E948CF34]
	(No symbol) [0x00007FF6E948D062]
	(No symbol) [0x00007FF6E947D3A3]
	BaseThreadInitThunk [0x00007FFDFD337034+20]
	RtlUserThreadStart [0x00007FFDFF1E2651+33]


In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--mute-audio")  # 음소거 옵션 추가
chrome_options.add_argument("--disable-audio-output")  # 오디오 출력 비활성화 옵션 추가

driver_path = "C:/Users/JY/chromedriver-win64/chromedriver"
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)

driver.get("https://www.melon.com/chart/index.htm")

# 각 아티스트 정보를 저장할 리스트
artist_info_list = []

# 각 곡의 상세 정보를 크롤링
for idx, song_detail in enumerate(song_detail_list[:22], 1):
    # 가수 정보가 있는 경우에만 진행
    if '가수' in song_detail:
        artist_name = song_detail['가수']

        # 가수 상세 페이지로 이동
        artist_link = driver.find_element_by_partial_link_text(artist_name)
        if artist_link:
            artist_link.click()
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "dl.atist_info.clfix")))
            
            # 아티스트 정보 크롤링
            artist_info = {}

            try:
                # 아티스트 고유 ID 추출
                artist_id_match = re.search(r"goArtistDetail\('(\d+)", driver.page_source)
                if artist_id_match:
                    artist_id = artist_id_match.group(1)
                    print(f"{artist_name}의 아티스트 ID: {artist_id}")
                    artist_info['아티스트 ID'] = artist_id
                else:
                    print(f"{artist_name}의 아티스트 ID를 찾을 수 없습니다.")
                    artist_info['아티스트 ID'] = 'N/A'
                
                debut_date_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='데뷔']/following-sibling::dd/span[@class='gubun']"))
                )
                debut_date = debut_date_element.text.strip()
                print(f"데뷔 날짜: {debut_date}")
                
                # 데뷔 날짜를 아티스트 정보에 추가
                artist_info['데뷔 날짜'] = debut_date
            except Exception as e:
                artist_info['아티스트 ID'] = 'N/A'
                artist_info['데뷔 날짜'] = 'N/A'
                print(f"데뷔일: N/A")

            try:
                activity_type_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='활동유형']/following-sibling::dd"))
                )
                activity_type = activity_type_element.text.strip()
                print(f"활동 유형: {activity_type}")
                
                # 활동 유형을 아티스트 정보에 추가
                artist_info['활동 유형'] = activity_type
            except Exception as e:
                print(f"활동 유형을 가져오는 중 에러 발생: {e}")
                artist_info['활동 유형'] = 'N/A'

            try:
                agency_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='소속사']/following-sibling::dd"))
                )
                agency = agency_element.text.strip()
                print(f"소속사: {agency}")
                
                # 소속사를 아티스트 정보에 추가
                artist_info['소속사'] = agency
            except Exception as e:
                print(f"소속사를 가져오는 중 에러 발생: {e}")
                artist_info['소속사'] = 'N/A'
                
            # 결과를 가로로 출력
            print(f"데뷔 날짜: {artist_info.get('데뷔 날짜', 'N/A')}, 활동 유형: {artist_info.get('활동 유형', 'N/A')}, 아티스트 ID: {artist_info.get('아티스트 ID', 'N/A')}, 소속사: {artist_info.get('소속사', 'N/A')}")
            
            # 각 아티스트 정보를 리스트에 추가
            artist_info_list.append(artist_info)

            disable_player_script = """
            var playButtons = document.querySelectorAll('a[href^="javascript:melon.play.playSong"]'); 
            playButtons.forEach(function(button) {
                var songIdMatch = button.getAttribute('href').match(/\('(\d+)'/);
                if (songIdMatch && (songIdMatch[1] === '36699489' || songIdMatch[1] === '36595401')) {
                    button.removeAttribute('href');
                    button.style.pointerEvents = 'none';
                }
            });
            """
            driver.execute_script(disable_player_script)

            # 아티스트 페이지에서 뒤로 돌아가기
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "service_list_song")))

# 전체 결과를 출력
for artist_info in artist_info_list:
    print(artist_info)

C:\Users\JY\AppData\Local\Temp/ipykernel_16756/2660852385.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
C:\Users\JY\AppData\Local\Temp/ipykernel_16756/2660852385.py:26: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  artist_link = driver.find_element_by_partial_link_text(artist_name)


LE SSERAFIM (르세라핌)의 아티스트 ID: 3092950
데뷔 날짜: 2022.05.02
활동 유형: 그룹
소속사: (주)쏘스뮤직
데뷔 날짜: 2022.05.02, 활동 유형: 그룹, 아티스트 ID: 3092950, 소속사: (주)쏘스뮤직
aespa의 아티스트 ID: 2899555
데뷔 날짜: 2020.11.17
활동 유형: 그룹
소속사: (주)SM엔터테인먼트
데뷔 날짜: 2020.11.17, 활동 유형: 그룹, 아티스트 ID: 2899555, 소속사: (주)SM엔터테인먼트
IVE (아이브)의 아티스트 ID: 3055146
데뷔 날짜: 2021.12.01
활동 유형: 그룹
소속사: 스타쉽 엔터테인먼트
데뷔 날짜: 2021.12.01, 활동 유형: 그룹, 아티스트 ID: 3055146, 소속사: 스타쉽 엔터테인먼트
태연 (TAEYEON)의 아티스트 ID: 236797
데뷔 날짜: 2015.10.07
활동 유형: 솔로
소속사: (주)SM엔터테인먼트
데뷔 날짜: 2015.10.07, 활동 유형: 솔로, 아티스트 ID: 236797, 소속사: (주)SM엔터테인먼트
정국의 아티스트 ID: 725987
데뷔일: N/A
활동 유형: 솔로
소속사: 빅히트엔터테인먼트
데뷔 날짜: N/A, 활동 유형: 솔로, 아티스트 ID: N/A, 소속사: 빅히트엔터테인먼트
AKMU (악뮤)의 아티스트 ID: 712452
데뷔 날짜: 2014.04.07
활동 유형: 그룹
소속사: (주)YG엔터테인먼트
데뷔 날짜: 2014.04.07, 활동 유형: 그룹, 아티스트 ID: 712452, 소속사: (주)YG엔터테인먼트
제니 (JENNIE)의 아티스트 ID: 995173
데뷔일: N/A
활동 유형: 솔로
소속사: (주)YG엔터테인먼트
데뷔 날짜: N/A, 활동 유형: 솔로, 아티스트 ID: N/A, 소속사: (주)YG엔터테인먼트
박재정의 아티스트 ID: 738480
데뷔 날짜: 2013.11.20
활동 유형: 솔로
소속사: 로맨틱 팩토리
데뷔 날짜: 2013.11.20, 활동 유형: 솔로,

In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--mute-audio")  # 음소거 옵션 추가
chrome_options.add_argument("--disable-audio-output")  # 오디오 출력 비활성화 옵션 추가

driver_path = "C:/Users/JY/chromedriver-win64/chromedriver"
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)

driver.get("https://www.melon.com/chart/index.htm")

# 각 아티스트 정보를 저장할 리스트
artist_info_list = []

# 각 곡의 상세 정보를 크롤링
for idx, song_detail in enumerate(song_detail_list[24:41], 1):
    # 가수 정보가 있는 경우에만 진행
    if '가수' in song_detail:
        artist_name = song_detail['가수']

        # 가수 상세 페이지로 이동
        artist_link = driver.find_element_by_partial_link_text(artist_name)
        if artist_link:
            artist_link.click()
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "dl.atist_info.clfix")))
            
            # 아티스트 정보 크롤링
            artist_info = {}

            try:
                # 아티스트 고유 ID 추출
                artist_id_match = re.search(r"goArtistDetail\('(\d+)", driver.page_source)
                if artist_id_match:
                    artist_id = artist_id_match.group(1)
                    print(f"{artist_name}의 아티스트 ID: {artist_id}")
                    artist_info['아티스트 ID'] = artist_id
                else:
                    print(f"{artist_name}의 아티스트 ID를 찾을 수 없습니다.")
                    artist_info['아티스트 ID'] = 'N/A'
                
                debut_date_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='데뷔']/following-sibling::dd/span[@class='gubun']"))
                )
                debut_date = debut_date_element.text.strip()
                print(f"데뷔 날짜: {debut_date}")
                
                # 데뷔 날짜를 아티스트 정보에 추가
                artist_info['데뷔 날짜'] = debut_date
            except Exception as e:
                artist_info['아티스트 ID'] = 'N/A'
                artist_info['데뷔 날짜'] = 'N/A'
                print(f"데뷔일: N/A")

            try:
                activity_type_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='활동유형']/following-sibling::dd"))
                )
                activity_type = activity_type_element.text.strip()
                print(f"활동 유형: {activity_type}")
                
                # 활동 유형을 아티스트 정보에 추가
                artist_info['활동 유형'] = activity_type
            except Exception as e:
                print(f"활동 유형을 가져오는 중 에러 발생: {e}")
                artist_info['활동 유형'] = 'N/A'

            try:
                agency_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='소속사']/following-sibling::dd"))
                )
                agency = agency_element.text.strip()
                print(f"소속사: {agency}")
                
                # 소속사를 아티스트 정보에 추가
                artist_info['소속사'] = agency
            except Exception as e:
                print(f"소속사를 가져오는 중 에러 발생: {e}")
                artist_info['소속사'] = 'N/A'
                
            # 결과를 가로로 출력
            print(f"데뷔 날짜: {artist_info.get('데뷔 날짜', 'N/A')}, 활동 유형: {artist_info.get('활동 유형', 'N/A')}, 아티스트 ID: {artist_info.get('아티스트 ID', 'N/A')}, 소속사: {artist_info.get('소속사', 'N/A')}")
            
            # 각 아티스트 정보를 리스트에 추가
            artist_info_list.append(artist_info)

            disable_player_script = """
            var playButtons = document.querySelectorAll('a[href^="javascript:melon.play.playSong"]'); 
            playButtons.forEach(function(button) {
                var songIdMatch = button.getAttribute('href').match(/\('(\d+)'/);
                if (songIdMatch && (songIdMatch[1] === '36699489' || songIdMatch[1] === '36595401')) {
                    button.removeAttribute('href');
                    button.style.pointerEvents = 'none';
                }
            });
            """
            driver.execute_script(disable_player_script)

            # 아티스트 페이지에서 뒤로 돌아가기
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "service_list_song")))

# 전체 결과를 출력
for artist_info in artist_info_list:
    print(artist_info)

C:\Users\JY\AppData\Local\Temp/ipykernel_16756/2142569740.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
C:\Users\JY\AppData\Local\Temp/ipykernel_16756/2142569740.py:26: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  artist_link = driver.find_element_by_partial_link_text(artist_name)


디오 (D.O.)의 아티스트 ID: 672853
데뷔 날짜: 2021.07.26
활동 유형: 솔로
소속사: (주)SM엔터테인먼트
데뷔 날짜: 2021.07.26, 활동 유형: 솔로, 아티스트 ID: 672853, 소속사: (주)SM엔터테인먼트
임영웅의 아티스트 ID: 994944
데뷔 날짜: 2016.08.08
활동 유형: 솔로
소속사: 물고기뮤직
데뷔 날짜: 2016.08.08, 활동 유형: 솔로, 아티스트 ID: 994944, 소속사: 물고기뮤직
Mariah Carey의 아티스트 ID: 2274
데뷔 날짜: 1990.06.12
활동 유형: 솔로
소속사: Epic
데뷔 날짜: 1990.06.12, 활동 유형: 솔로, 아티스트 ID: 2274, 소속사: Epic
Red Velvet (레드벨벳)의 아티스트 ID: 780066
데뷔 날짜: 2014.08.01
활동 유형: 그룹
소속사: (주)SM엔터테인먼트
데뷔 날짜: 2014.08.01, 활동 유형: 그룹, 아티스트 ID: 780066, 소속사: (주)SM엔터테인먼트
임영웅의 아티스트 ID: 994944
데뷔 날짜: 2016.08.08
활동 유형: 솔로
소속사: 물고기뮤직
데뷔 날짜: 2016.08.08, 활동 유형: 솔로, 아티스트 ID: 994944, 소속사: 물고기뮤직
NewJeans의 아티스트 ID: 3114174
데뷔 날짜: 2022.07.22
활동 유형: 그룹
소속사: 주식회사 어도어
데뷔 날짜: 2022.07.22, 활동 유형: 그룹, 아티스트 ID: 3114174, 소속사: 주식회사 어도어
(여자)아이들의 아티스트 ID: 2137482
데뷔 날짜: 2018.05.02
활동 유형: 그룹
소속사: 큐브엔터테인먼트
데뷔 날짜: 2018.05.02, 활동 유형: 그룹, 아티스트 ID: 2137482, 소속사: 큐브엔터테인먼트
NewJeans의 아티스트 ID: 3114174
데뷔 날짜: 2022.07.22
활동 유형: 그룹
소속사: 주식회사 어도어
데뷔 날짜: 2022.07.22, 활동 유형: 그룹, 아티스

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--mute-audio")  # 음소거 옵션 추가
chrome_options.add_argument("--disable-audio-output")  # 오디오 출력 비활성화 옵션 추가

driver_path = "C:/Users/JY/chromedriver-win64/chromedriver"
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)

driver.get("https://www.melon.com/chart/index.htm")

# 각 아티스트 정보를 저장할 리스트
artist_info_list = []

# 각 곡의 상세 정보를 크롤링
for idx, song_detail in enumerate(song_detail_list[43:59], 1):
    # 가수 정보가 있는 경우에만 진행
    if '가수' in song_detail:
        artist_name = song_detail['가수']

        # 가수 상세 페이지로 이동
        artist_link = driver.find_element_by_partial_link_text(artist_name)
        if artist_link:
            artist_link.click()
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "dl.atist_info.clfix")))
            
            # 아티스트 정보 크롤링
            artist_info = {}

            try:
                # 아티스트 고유 ID 추출
                artist_id_match = re.search(r"goArtistDetail\('(\d+)", driver.page_source)
                if artist_id_match:
                    artist_id = artist_id_match.group(1)
                    print(f"{artist_name}의 아티스트 ID: {artist_id}")
                    artist_info['아티스트 ID'] = artist_id
                else:
                    print(f"{artist_name}의 아티스트 ID를 찾을 수 없습니다.")
                    artist_info['아티스트 ID'] = 'N/A'
                
                debut_date_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='데뷔']/following-sibling::dd/span[@class='gubun']"))
                )
                debut_date = debut_date_element.text.strip()
                print(f"데뷔 날짜: {debut_date}")
                
                # 데뷔 날짜를 아티스트 정보에 추가
                artist_info['데뷔 날짜'] = debut_date
            except Exception as e:
                artist_info['아티스트 ID'] = 'N/A'
                artist_info['데뷔 날짜'] = 'N/A'
                print(f"데뷔일: N/A")

            try:
                activity_type_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='활동유형']/following-sibling::dd"))
                )
                activity_type = activity_type_element.text.strip()
                print(f"활동 유형: {activity_type}")
                
                # 활동 유형을 아티스트 정보에 추가
                artist_info['활동 유형'] = activity_type
            except Exception as e:
                print(f"활동 유형을 가져오는 중 에러 발생: {e}")
                artist_info['활동 유형'] = 'N/A'

            try:
                agency_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='소속사']/following-sibling::dd"))
                )
                agency = agency_element.text.strip()
                print(f"소속사: {agency}")
                
                # 소속사를 아티스트 정보에 추가
                artist_info['소속사'] = agency
            except Exception as e:
                print(f"소속사를 가져오는 중 에러 발생: {e}")
                artist_info['소속사'] = 'N/A'
                
            # 결과를 가로로 출력
            print(f"데뷔 날짜: {artist_info.get('데뷔 날짜', 'N/A')}, 활동 유형: {artist_info.get('활동 유형', 'N/A')}, 아티스트 ID: {artist_info.get('아티스트 ID', 'N/A')}, 소속사: {artist_info.get('소속사', 'N/A')}")
            
            # 각 아티스트 정보를 리스트에 추가
            artist_info_list.append(artist_info)

            disable_player_script = """
            var playButtons = document.querySelectorAll('a[href^="javascript:melon.play.playSong"]'); 
            playButtons.forEach(function(button) {
                var songIdMatch = button.getAttribute('href').match(/\('(\d+)'/);
                if (songIdMatch && (songIdMatch[1] === '36699489' || songIdMatch[1] === '36595401')) {
                    button.removeAttribute('href');
                    button.style.pointerEvents = 'none';
                }
            });
            """
            driver.execute_script(disable_player_script)

            # 아티스트 페이지에서 뒤로 돌아가기
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "service_list_song")))

# 전체 결과를 출력
for artist_info in artist_info_list:
    print(artist_info)

C:\Users\JY\AppData\Local\Temp/ipykernel_16756/3806420525.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
C:\Users\JY\AppData\Local\Temp/ipykernel_16756/3806420525.py:26: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  artist_link = driver.find_element_by_partial_link_text(artist_name)


Ariana Grande의 아티스트 ID: 597451
데뷔 날짜: 2011.12.12
활동 유형: 솔로
소속사를 가져오는 중 에러 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6E95182B2+55298]
	(No symbol) [0x00007FF6E9485E02]
	(No symbol) [0x00007FF6E93405AB]
	(No symbol) [0x00007FF6E938175C]
	(No symbol) [0x00007FF6E93818DC]
	(No symbol) [0x00007FF6E93BCBC7]
	(No symbol) [0x00007FF6E93A20EF]
	(No symbol) [0x00007FF6E93BAAA4]
	(No symbol) [0x00007FF6E93A1E83]
	(No symbol) [0x00007FF6E937670A]
	(No symbol) [0x00007FF6E9377964]
	GetHandleVerifier [0x00007FF6E9890AAB+3694587]
	GetHandleVerifier [0x00007FF6E98E728E+4048862]
	GetHandleVerifier [0x00007FF6E98DF173+4015811]
	GetHandleVerifier [0x00007FF6E95B47D6+695590]
	(No symbol) [0x00007FF6E9490CE8]
	(No symbol) [0x00007FF6E948CF34]
	(No symbol) [0x00007FF6E948D062]
	(No symbol) [0x00007FF6E947D3A3]
	BaseThreadInitThunk [0x00007FFDFD337034+20]
	RtlUserThreadStart [0x00007FFDFF1E2651+33]

데뷔 날짜: 2011.12.12, 활동 유형: 솔로, 아티스트 ID: 597451, 소속사: N/A
임영웅의 아티스트 ID: 994944
데뷔 날짜: 2016.08.08
활동

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--mute-audio")  # 음소거 옵션 추가
chrome_options.add_argument("--disable-audio-output")  # 오디오 출력 비활성화 옵션 추가

driver_path = "C:/Users/JY/chromedriver-win64/chromedriver"
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)

driver.get("https://www.melon.com/chart/index.htm")

# ...

# 각 곡의 상세 정보를 크롤링
excluded_indices = [14,26,99]
for idx, song_detail in enumerate(song_detail_list, 1):
    # 특정 조건에서 제외할 index인 경우 건너뜀
    if idx in excluded_indices:
        continue

    # 가수 정보가 있는 경우에만 진행
    if '가수' in song_detail:
        artist_name = song_detail['가수']

        # 가수 상세 페이지로 이동
        artist_link = driver.find_element_by_partial_link_text(artist_name)
        if artist_link:
            artist_link.click()
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "dl.atist_info.clfix")))
            
            # 아티스트 정보 크롤링
            artist_info_dl = driver.find_element_by_css_selector("dl.atist_info.clfix")

            try:
                # 아티스트 고유 ID 추출
                artist_id_match = re.search(r"goArtistDetail\('(\d+)", driver.page_source)
                if artist_id_match:
                    artist_id = artist_id_match.group(1)

                    # 결과에 아티스트 ID, 아티스트 이름 추가
                    print(f"아티스트 ID: {artist_id}, 아티스트 이름: {artist_name}")
                    
                    song_detail['아티스트 ID'] = artist_id
                    song_detail['아티스트 이름'] = artist_name
                else:
                    print(f"{artist_name}의 아티스트 ID를 찾을 수 없습니다.")
                    song_detail['아티스트 ID'] = 'N/A'
                    song_detail['아티스트 이름'] = 'N/A'
                
                debut_date_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='데뷔']/following-sibling::dd/span[@class='gubun']"))
                )
                debut_date = debut_date_element.text.strip()
                print(f"데뷔 날짜: {debut_date}")
                
                # 데뷔 날짜를 song_detail에 추가
                song_detail['데뷔 날짜'] = debut_date
            except Exception as e:
                song_detail['아티스트 ID'] = 'N/A'
                song_detail['아티스트 이름'] = 'N/A'
                song_detail['데뷔 날짜'] = 'N/A'
                song_detail['소속사'] = 'N/A'
                print(f"데뷔일: N/A")

            try:
                activity_type_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='활동유형']/following-sibling::dd"))
                )
                activity_type = activity_type_element.text.strip()
                print(f"활동 유형: {activity_type}")
                
                # 활동 유형을 song_detail에 추가
                song_detail['활동 유형'] = activity_type
            except Exception as e:
                print(f"활동 유형을 가져오는 중 에러 발생: {e}")
                song_detail['활동 유형'] = 'N/A'

            try:
                agency_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//dt[text()='소속사']/following-sibling::dd"))
                )
                agency = agency_element.text.strip()
                print(f"소속사: {agency}")
                
                # 소속사를 song_detail에 추가
                song_detail['소속사'] = agency
            except Exception as e:
                song_detail['소속사'] = 'N/A'
                print(f"소속사: N/A")
                
            disable_player_script = """
            var playButtons = document.querySelectorAll('a[href^="javascript:melon.play.playSong"]'); 
            playButtons.forEach(function(button) {
                var songIdMatch = button.getAttribute('href').match(/\('(\d+)'/);
                if (songIdMatch && (songIdMatch[1] === '36699489' || songIdMatch[1] === '36595401')) {
                    button.removeAttribute('href');
                    button.style.pointerEvents = 'none';
                }
            });
            """
            driver.execute_script(disable_player_script)

            # 아티스트 페이지에서 뒤로 돌아가기
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "service_list_song")))

# 결과를 출력
for song_detail in song_detail_list:
    print(f"아티스트 ID: {song_detail.get('아티스트 ID', 'N/A')}, 아티스트 이름: {song_detail.get('아티스트 이름', 'N/A')}, 데뷔 날짜: {song_detail.get('데뷔 날짜', 'N/A')}, 활동 유형: {song_detail.get('활동 유형', 'N/A')}, 소속사: {song_detail.get('소속사', 'N/A')}")

C:\Users\JY\AppData\Local\Temp/ipykernel_11996/1792508353.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
C:\Users\JY\AppData\Local\Temp/ipykernel_11996/1792508353.py:30: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  artist_link = driver.find_element_by_partial_link_text(artist_name)
C:\Users\JY\AppData\Local\Temp/ipykernel_11996/1792508353.py:36: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  artist_info_dl = driver.find_element_by_css_selector("dl.atist_info.clfix")


아티스트 ID: 3092950, 아티스트 이름: LE SSERAFIM (르세라핌)
데뷔 날짜: 2022.05.02
활동 유형: 그룹
소속사: (주)쏘스뮤직
아티스트 ID: 2899555, 아티스트 이름: aespa
데뷔 날짜: 2020.11.17
활동 유형: 그룹
소속사: (주)SM엔터테인먼트
아티스트 ID: 3055146, 아티스트 이름: IVE (아이브)
데뷔 날짜: 2021.12.01
활동 유형: 그룹
소속사: 스타쉽 엔터테인먼트
아티스트 ID: 236797, 아티스트 이름: 태연 (TAEYEON)
데뷔 날짜: 2015.10.07
활동 유형: 솔로
소속사: (주)SM엔터테인먼트
아티스트 ID: 738480, 아티스트 이름: 박재정
데뷔 날짜: 2013.11.20
활동 유형: 솔로
소속사: 로맨틱 팩토리
아티스트 ID: 712452, 아티스트 이름: AKMU (악뮤)
데뷔 날짜: 2014.04.07
활동 유형: 그룹
소속사: (주)YG엔터테인먼트
아티스트 ID: 2189803, 아티스트 이름: 범진
데뷔 날짜: 2019.05.25
활동 유형: 솔로
소속사: JMG (더블엑스엔터테인먼트)
아티스트 ID: 725987, 아티스트 이름: 정국
데뷔일: N/A
활동 유형: 솔로
소속사: 빅히트엔터테인먼트
아티스트 ID: 995173, 아티스트 이름: 제니 (JENNIE)
데뷔일: N/A
활동 유형: 솔로
소속사: (주)YG엔터테인먼트
아티스트 ID: 712452, 아티스트 이름: AKMU (악뮤)
데뷔 날짜: 2014.04.07
활동 유형: 그룹
소속사: (주)YG엔터테인먼트
아티스트 ID: 724619, 아티스트 이름: EXO
데뷔 날짜: 2012.04.08
활동 유형: 그룹
소속사: (주)SM엔터테인먼트
아티스트 ID: 3055146, 아티스트 이름: IVE (아이브)
데뷔 날짜: 2021.12.01
활동 유형: 그룹
소속사: 스타쉽 엔터테인먼트
아티스트 ID: 3114174, 아티스트 이름: NewJeans
데뷔 날짜: 2022.07.22
활동 유형: 그룹


아티스트 ID: 2645970, 아티스트 이름: 최유리, 데뷔 날짜: 2020.02.24, 활동 유형: 솔로, 소속사: 쇼파르엔터테인먼트
아티스트 ID: 994944, 아티스트 이름: 임영웅, 데뷔 날짜: 2016.08.08, 활동 유형: 솔로, 소속사: 물고기뮤직
아티스트 ID: 751611, 아티스트 이름: 헤이즈 (Heize), 데뷔 날짜: 2014.01.17, 활동 유형: 솔로, 소속사: 피네이션
아티스트 ID: 672375, 아티스트 이름: 방탄소년단, 데뷔 날짜: 2013.06.13, 활동 유형: 그룹, 소속사: 빅히트엔터테인먼트
아티스트 ID: 994944, 아티스트 이름: 임영웅, 데뷔 날짜: 2016.08.08, 활동 유형: 솔로, 소속사: 물고기뮤직
아티스트 ID: 2743730, 아티스트 이름: The Kid LAROI, 데뷔 날짜: 2019.08.21, 활동 유형: 솔로, 소속사: N/A
아티스트 ID: 752425, 아티스트 이름: 폴킴, 데뷔 날짜: 2014.01.21, 활동 유형: 솔로, 소속사: N/A
아티스트 ID: 967229, 아티스트 이름: 릴러말즈 (Leellamarz), 데뷔 날짜: 2017.07.07, 활동 유형: 솔로, 소속사: N/A
아티스트 ID: 2758756, 아티스트 이름: BE'O (비오), 데뷔 날짜: 2020.09.05, 활동 유형: 솔로, 소속사: BigPlanetMade, 페임어스 엔터테인먼트
아티스트 ID: 2645970, 아티스트 이름: 최유리, 데뷔 날짜: 2020.02.24, 활동 유형: 솔로, 소속사: 쇼파르엔터테인먼트
아티스트 ID: 3064111, 아티스트 이름: H1-KEY (하이키), 데뷔 날짜: 2022.01.05, 활동 유형: 그룹, 소속사: 주식회사 그랜드라인그룹
아티스트 ID: N/A, 아티스트 이름: N/A, 데뷔 날짜: N/A, 활동 유형: 솔로, 소속사: (주)YG엔터테인먼트
아티스트 ID: N/A, 아티스트 이름: N/A, 데뷔 날짜: N/A, 활동 유형: 솔로, 소속사

In [18]:
import csv

# 결과를 저장할 CSV 파일 경로
csv_file_path = "C:/Users/JY/Desktop/bigdaeseol/artist_details.csv"

# 결과를 CSV 파일에 쓰기
with open(csv_file_path, mode='w', encoding='utf-8', newline='') as csv_file:
    # CSV 파일에 쓸 헤더 작성
    fieldnames = ['아티스트 ID', '아티스트 이름', '데뷔 날짜', '활동 유형', '소속사']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    # 결과를 CSV 파일에 쓰기
    for song_detail in song_detail_list:
        writer.writerow({
            '아티스트 ID': song_detail.get('아티스트 ID', 'N/A'),
            '아티스트 이름': song_detail.get('아티스트 이름', 'N/A'),
            '데뷔 날짜': song_detail.get('데뷔 날짜', 'N/A'),
            '활동 유형': song_detail.get('활동 유형', 'N/A'),
            '소속사': song_detail.get('소속사', 'N/A'),
        })

print(f"결과가 {csv_file_path}에 저장되었습니다.")


결과가 C:/Users/JY/Desktop/bigdaeseol/artist_details.csv에 저장되었습니다.
